In [198]:
# Import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd
import time

In [5]:
# Initialize Pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
# Define database and collection
db = client.NAME
collection = db.items

In [100]:
# Get splinter ready
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [65]:
'''Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables that you can reference later.'''

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page & parse with BeautifulSoup
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [76]:
news_title = soup.find('div', class_='content_title').text
news_para = soup.find('div', class_="rollover_description_inner").text

# This should actually work with 'article_teaser_body' but it won't load!

## JPL Mars Space Images - Featured Image

In [93]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called featured_image_url.
s = soup.find('article', class_='carousel_item')['style']
url = s.split("'")[1]
featured_image_url = 'https://www.jpl.nasa.gov' + url

## Mars Weather (Twitter)

In [141]:
'''
Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
Save the tweet text for the weather report as a variable called mars_weather.
'''

url = 'https://twitter.com/marswxreport?lang=en'

html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [151]:
mars_weather = soup.find('div', class_='js-tweet-text-container').text
mars_weather = mars_weather.strip()
mars_weather = mars_weather.strip('pic.twitter.com/QTKm4klPNr')

In [153]:
print(mars_weather)

InSight sol 448 (2020-02-29) low -94.1ºC (-137.3ºF) high -8.3ºC (17.0ºF)
winds from the SSW at 5.5 m/s (12.4 mph) gusting to 19.9 m/s (44.6 mph)
pressure at 6.30 hPa


## Mars Facts

In [178]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
r = requests.get("https://space-facts.com/mars/")
soup = BeautifulSoup(r.content,'lxml')
table = soup.find('table', id='tablepress-p-mars-no-2')
df = pd.read_html(str(table))
df = df[0]

df.columns = ['description', 'value']
df.set_index(keys=['description'], inplace=True)

# Use Pandas to convert the data to a HTML table string.
html = df.to_html()
html_table = html.replace('\n', '')

## Mars Hemispheres

In [213]:
r = requests.get("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
soup = BeautifulSoup(r.content,'lxml')

In [214]:
links = soup.find_all('a', class_='itemLink')

In [236]:
hemisphere_image_urls = []

for link in links:
    hemisphere = {}
    r = requests.get("https://astrogeology.usgs.gov/" + link['href'])
    soup = BeautifulSoup(r.content,'lxml')
    l = soup.find('li')
    link = l.contents[0]
    img_url = link['href']
    title = soup.title.text
    title = title.split("|")[0]
    hemisphere["title"] = title
    hemisphere["img_url"] = img_url
    hemisphere_image_urls.append(hemisphere)
    time.sleep(1)

In [237]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]